In [2]:
import cv2
import numpy as np
import pandas as pd
import os
import shutil
import random
import torch
from PIL import Image

In [3]:
flower_label = {
                'buquet' : 0,
                'rose' : 0,             # 장미
                'empty1' : 1,
                'tulip' : 2,            # 튤립
                'carnation' : 3,
                'empty2' : 4,        # 카네이션
                'lily' : 5,             # 백합
                'lisianthus' : 6,       # 리시안셔스
                'small_chrysanthemum' : 7,    # 소국
                'pompom_blooms' : 8,    # 퐁퐁국화
                'sunflower': 9,         # 해바라기
                'gerbera' : 10,          # 거베라
                'alstroemeria' : 11,     # 알스트로메리아
                'hydrangea' : 12,       # 수국
                'peony' : 13,           # 작약
                'stock' : 14,           # 스톡
                'freesia' : 15,         # 프리지아
                'ranunculus' : 16,      # 라넌큘러스
                'empty3' : 17,
                'butterfly':18,  # 라넌큘러스 버터플라이
                'cala': 19,               # 칼라 꽃
                'snapdragon' : 20,        # 금어초
                }

flower_name_list = [i[0] for i in sorted([[k,v] for k,v in flower_label.items()], key=lambda x: x[1])]

flower = 'peony'

f_num = flower_label[flower]


# 모델 로드

model = torch.hub.load('ultralytics/yolov5', 'custom', path=f'./models/{flower}.pt', force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\SSAFY/.cache\torch\hub\master.zip
YOLOv5  2023-5-9 Python-3.11.2 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 Super with Max-Q Design, 8192MiB)



requirements: C:\Users\SSAFY\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
one_YOLOv5m summary: 212 layers, 20905467 parameters, 0 gradients, 48.0 GFLOPs
Adding AutoShape... 


## 모델로 바운딩 박스 그리기


### (optional) 재학습 시, 1과 4의 데이터들을 삭제


In [48]:
labels = os.listdir(f'./1_labels_data/{flower}')
model_images = os.listdir(f'./4_model_images/{flower}')

for i in labels:
    os.remove(f'./1_labels_data/{flower}/{i}')
    
for i in model_images:
    os.remove(f'./4_model_images/{flower}/{i}')

### 바운딩 박스 그리기


In [49]:
image_list = os.listdir(f'./0_flower_data/{flower}')

if not os.path.isdir(f'./4_model_images/{flower}'):
    os.mkdir(f'./4_model_images/{flower}')

for f_name in image_list[:]:
    
    try:
        img = Image.open(f'./0_flower_data/{flower}/{f_name}')
    except:
        os.remove(f'./0_flower_data/{flower}/{f_name}')
        continue
    
    results = model(img)
    
    if len(results.xywh[0]):
        del_idx = []
        xywhn_list = results.xywhn[0]
        for i in range(len(xywhn_list)-1):
            src = xywhn_list[i]
            for j in range(i+1, len(xywhn_list)):
                tar = xywhn_list[j]
                
                if abs(src[0] - tar[0]) <= 0.05 and abs(src[1] - tar[1]) <= 0.05:
                    if src[4] > tar[4]:
                        del_idx.append(j)
                    else:
                        del_idx.append(i)
            if src[4] < 0.3:
                del_idx.append(i)
        else:
            if xywhn_list[-1][4] < 0.3:
                del_idx.append(i+1)
            del_idx = sorted(list(set(del_idx)), reverse=True)
            for di in del_idx:
                if di == len(results.pred[0]):
                    results.xywhn[0] = results.xywhn[0][:di, :]
                    results.pred[0] = results.pred[0][:di, :]
                else:
                    results.xywhn[0] = torch.cat((results.xywhn[0][:di, :], results.xywhn[0][di+1:, :]), dim=0)
                    results.pred[0] = torch.cat((results.pred[0][:di, :], results.pred[0][di+1:, :]), dim=0)

        for result in results.xywhn[0]:
            x,y,w,h,score,CLS = map(float, result)
            CLS = int(CLS)
        
            with open(f'./1_labels_data/{flower}/{f_name[:-4]}.txt', 'a') as txt:
                txt.write(f'{CLS} {round(x, 4)} {round(y, 4)} {round(w, 4)} {round(h, 4)}\n')
    else:
        continue
    
    results.save()
    
    shutil.move(f'./runs/detect/exp/{f_name}', f'./4_model_images/{flower}/')
    os.rmdir('./runs/detect/exp')

os.rmdir('./runs/detect')

Saved 1 image to runs\detect\exp2
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp
Saved 1 i

OSError: [WinError 145] 디렉터리가 비어 있지 않습니다: './runs/detect'

## 데이터 수작업으로 판별해주기

In [5]:
# 마우스 이벤트 콜백 함수
def mouse_callback(event, x, y, flags, param):
    global point1, point2, cnt, img, coordi, f_label_name
    if event == cv2.EVENT_LBUTTONDOWN:
        if cnt%2:
            img_copy = img.copy()
            point1 = (x, y)
            cv2.line(img_copy, (x, y), (x, img.shape[0]), (0, 255, 0), 5)
            cv2.line(img_copy, (x, y), (img.shape[1], y), (0, 255, 0), 5)
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
        else:
            px, py = point1
            point2 = (x, y)
            
            cv2.line(img, (px, py), (px, y), (0, 255, 0), 5)
            cv2.line(img, (px, py), (x, py), (0, 255, 0), 5)
            cv2.line(img, (x, y), (px, y), (0, 255, 0), 5)
            cv2.line(img, (x, y), (x, py), (0, 255, 0), 5)
            
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
            
            if point1 and point2:
                # 좌측상단 좌표와 우측하단 좌표로 변환
                x1, y1 = min(point1[0], point2[0]), min(point1[1], point2[1])
                x2, y2 = max(point1[0], point2[0]), max(point1[1], point2[1])
                
                # yolo의 바운딩박스 수치로 변환
                x = round((x1+(x2-x1)/2)/img.shape[1], 4)
                y = round((y1+(y2-y1)/2)/img.shape[0], 4)
                w = round((x2-x1)/img.shape[1], 4)
                h = round((y2-y1)/img.shape[0], 4)
                
                coordi += f"{f_num} {x} {y} {w} {h}\n"
                # 좌표 저장
            
        cnt += 1

def update_text(image, f_label_name):
    font = cv2.FONT_HERSHEY_SIMPLEX
    x, y = image.shape[:2]
    font_scale = (x+y)//1000
    font_color = (255, 255, 255)
    line_type = (x+y)//200
    position = [x//10,y//10]
    text = f'{f_label_name} : {flower_label[f_label_name]}'
    cv2.putText(image, text, position, font, font_scale, font_color, line_type)
    font_color = (0, 0, 0)
    line_type = (x+y)//600
    cv2.putText(image, text, position, font, font_scale, font_color, line_type)


break_plag = False
flower_list = os.listdir(f'./4_model_images/{flower}')

if not os.path.isdir(f'./1_labels_data/{flower}'):
    os.mkdir(f'./1_labels_data/{flower}')

f_idx = 0

while 1:

    f_name = flower_list[f_idx]
    f_num = flower_label[flower]
    
    f_name_idx = flower_name_list.index(flower)
    f_label_name = flower

    if break_plag:
        break
    
    # 이미지 파일 경로
    img_path = f'./4_model_images/{flower}/{f_name}'

    # 이미지 로드 및 윈도우 생성
    img = cv2.imread(img_path)
    
    if type(img) != np.ndarray:
        os.remove(f'./4_model_images/{flower}/{f_name}')
        continue
    
    height, width = img.shape[:2]
    
    if height > width:
        new_height = 640
        new_width = int(width * (new_height / height))
    else:
        new_width = 640
        new_height = int(height * (new_width / width))
    
    cv2.namedWindow(f'{f_name} {width} * {height}', cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f'{f_name} {width} * {height}', (new_width, new_height))
    cv2.setMouseCallback(f'{f_name} {width} * {height}', mouse_callback)
    
    img_copy = img.copy()
    update_text(img_copy, f_label_name)
    
    cv2.imshow(f'{f_name} {width} * {height}', img_copy)

    # 마우스 이벤트를 통한 좌표 저장
    point1, point2, coordi, cnt = None, None, '', 1
    while True:
        key = cv2.waitKey(1) & 0xFF
        
        # 좌표를 txt 파일로 저장하고 다음으로 넘어가기
        # 만약 좌표를 그리지 않았다면 저장하지 않고 그냥 넘어가기
        if key == ord("q"):
            f_idx += 1
            if coordi == '':
                break
            
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "a") as f:
                f.write(coordi)
            break

        # 다음 꽃 바운딩
        elif key == ord('e'):
            cnt = 1
            f_name_idx = f_name_idx+1 if len(flower_name_list)-1 > f_name_idx else 0
            f_label_name = flower_name_list[f_name_idx]
            f_num = flower_label.get(f_label_name)

            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
        # 이전 꽃 바운딩
        elif key == ord('w'):
            cnt = 1
            f_name_idx = f_name_idx-1 if 0 < f_name_idx else len(flower_name_list)-1
            f_label_name = flower_name_list[f_name_idx]
            f_num = flower_label.get(f_label_name)
            
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
            
        # 저장된 좌표까지 전부 지우기
        elif key == ord('m'):
            cnt = 1
            with open(f"./1_labels_data/{flower}/{f_name[:-4]}.txt", "w") as f:
                f.write('')


        # 저장하지 않은 좌표들을 지우고, 다시 그리기
        elif key == ord('c'):
            cnt = 1
            img = cv2.imread(img_path)
            img_copy = img.copy()
            update_text(img_copy, f_label_name)
            coordi = ''
            cv2.imshow(f'{f_name} {width} * {height}', img_copy)
        
        # 볼 것도 없다. 데이터셋에서 그냥 삭제
        elif key == ord('f'):
            os.remove(f'./4_model_images/{flower}/{f_name}')
            flower_list = os.listdir(f'./4_model_images/{flower}')
            break
        
        # 4번 뿐만 아니라, 원본까지 날려버리기
        elif key == ord('g'):
            os.remove(f'./4_model_images/{flower}/{f_name}')
            os.remove(f'./0_flower_data/{flower}/{f_name}')
            flower_list = os.listdir(f'./4_model_images/{flower}')
            break
        
        # 지나쳐버렸다... 앞으로 돌아가기
        elif key == ord('z'):
            if f_idx > 0:
                f_idx -= 1
                break
            elif f_idx == 0:
                pass
        
        # 여기서 끝내기!
        elif key == ord('p'):
            break_plag = True
            break

    # 윈도우 종료 및 리소스 해제
    cv2.destroyAllWindows()

### (optional) 한 번 더 학습 시키고 싶다!


In [18]:
# 3의 이전 파일들 삭제

for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

# 4에 존재하는 이미지의 이름을 비교해, 0과 1의 파일을 3으로 보내기!

bound_img_list = os.listdir(f'./4_model_images/{flower}')

random_list = random.sample([i for i in range(len(bound_img_list))], len(bound_img_list)//10*2)

for i in range(len(bound_img_list)):
    CLS = 'train'
    img = bound_img_list[i]
    txt = f'{img[:-4]}.txt'
    
    if i in random_list:
        CLS = 'test'

    if not os.path.isfile(f'./0_flower_data/{flower}/{img}'):
        continue
        
    if not os.path.isfile(f'./1_labels_data/{flower}/{txt}'):
        os.remove(f'./4_model_images/{flower}/{img}')
        os.remove(f'./0_flower_data/{flower}/{img}')
        continue
    try:
        shutil.copy(f'./0_flower_data/{flower}/{img}', f'./3_one_time_data/{CLS}/images/{img}')
        shutil.copy(f'./1_labels_data/{flower}/{txt}', f'./3_one_time_data/{CLS}/labels/{txt}')
    except:
        print(f'{img}')

### (optional) 4와 0을 맞추고 싶다!


In [6]:
if input('realy?') == 'y':
    pass
else:
    raise Exception('어림도 없다 암!')

img_0 = os.listdir(f'./0_flower_data/{flower}')
img_4 = os.listdir(f'./4_model_images/{flower}')

for i in img_0:
    if i not in img_4:
        os.remove(f'./0_flower_data/{flower}/{i}')

### (optional) 4에 따라 1을 지우고 싶다!

In [11]:
if input('realy?') == 'y':
    pass
else:
    raise Exception('어림도 없다 암!')

img_1 = os.listdir(f'./1_labels_data/{flower}')
img_4 = os.listdir(f'./4_model_images/{flower}')

for i in img_1:
    if i[:-4]+'.jpg' not in img_4:
        os.remove(f'./1_labels_data/{flower}/{i[:-4]}.txt')

## 결측치, 이상치를 제거한 후, 1과 4의 데이터를 5로 옮기자!


In [6]:
import random

bound_img_list = os.listdir(f'./4_model_images/{flower}')

random_list = random.sample([i for i in range(len(bound_img_list))], len(bound_img_list)//10*2)

for i in range(len(bound_img_list)):
    CLS = 'train'
    img = bound_img_list[i]
    txt = f'{img[:-4]}.txt'
    
    if i in random_list:
        CLS = 'test'
        
    if not os.path.isfile(f'./0_flower_data/{flower}/{img}'):
        continue
        
    if not os.path.isfile(f'./1_labels_data/{flower}/{txt}'):
        os.remove(f'./4_model_images/{flower}/{img}')
        os.remove(f'./0_flower_data/{flower}/{img}')
        continue
    try:
        shutil.move(f'./0_flower_data/{flower}/{img}', f'./5_total_data/{CLS}/images/{img}')
        shutil.move(f'./1_labels_data/{flower}/{txt}', f'./5_total_data/{CLS}/labels/{txt}')
    except:
        print(f'{img}')

python train.py --img 640 --rect --batch 16 --epochs 1 --data total_data.yaml --cfg total_yolov5m.yaml --weight yolov5m.pt --name {flower}_test


### 남은 잔여 파일들 정리


In [ ]:
# remain_img = os.listdir(f'./0_flower_data/{flower}')

# for i in range(len(remain_img)):
#     os.remove(f'./0_flower_data/{flower}/{remain_img[i]}')

remain_txt = os.listdir(f'./1_labels_data/{flower}')

for i in range(len(remain_txt)):
    os.remove(f'./1_labels_data/{flower}/{remain_txt[i]}')

remain_bound_img = os.listdir(f'./4_model_images/{flower}')

for i in remain_bound_img:
    os.remove(f'./4_model_images/{flower}/{i}')

### 3의 이전 파일들 삭제하기


In [19]:
for C in ['train', 'test']:
    for L in ['labels', 'images']:
        for i in os.listdir(f'./3_one_time_data/{C}/{L}'):
            os.remove(f'./3_one_time_data/{C}/{L}/{i}')
    
    if os.path.isfile(f'./3_one_time_data/{C}/labels.cache'):
        os.remove(f'./3_one_time_data/{C}/labels.cache')

### 5의 데이터세트 중, 한 종류를 3으로 옮기고 싶을 때


In [79]:
for C in ['train', 'test']:
    for L in ['images', 'labels']:
        f_list = os.listdir(f'./5_total_data/{C}/{L}')
        
        for f in f_list:
            if f[0:2] == 'ca':
                shutil.move(f'./5_total_data/{C}/{L}/{f}', f'./3_one_time_data/{C}/{L}/{f}')

### (optional) 라벨에 중복값이 있는 경우 삭제해주자


In [73]:
for CLS in ['train', 'test']:
    for f_name in os.listdir(f'./3_one_time_data/{CLS}/labels'):
        txt = []
        del_idx = []
        with open(f'./3_one_time_data/{CLS}/labels/{f_name}', 'r') as f:
            for x in f.readlines():
                txt.append(x)
            
            for i in range(len(txt)-1):
                for j in range(i+1, len(txt)):
                    if txt[i] == txt[j]:
                        del_idx.append(i)
        del_idx = sorted(del_idx, reverse=True)
        for i in del_idx:
            txt.pop(i)
        
        txt = ''.join(txt)
        
        with open(f'./3_one_time_data/{CLS}/labels/{f_name}', 'w') as f:
            f.write(txt)

### (주의!) 5의 데이터 세트 중, 한 종류를 지우고 싶을 때


In [31]:
if input('realy?') == 'y':
    pass
else:
    raise Exception('어림도 없다 암!')


for C in ['train', 'test']:
    for L in ['images', 'labels']:
        f_list = os.listdir(f'./5_total_data/{C}/{L}')
        
        for f in f_list:
            if f[0] == 'c':
                os.remove(f'./5_total_data/{C}/{L}/{f}')

### (주의!) 0의 원본 데이터 삭제


In [ ]:
if input('realy?') == 'y':
    pass
else:
    raise Exception('어림도 없다 암!')

img_list = os.listdir(f'./0_flower_data/{flower}')

for i in img_list:
    os.remove(f'./0_flower_data/{flower}/{i}')

In [21]:
for CLS in ['train', 'test']:
    images_list = {i:True for i in os.listdir(f'./5_total_data/{CLS}/images')}
    labels_list = os.listdir(f'./5_total_data/{CLS}/labels')
    
    for i in labels_list:
        if not images_list.get(i[:-4]+'.jpg'):
            shutil.move(f'./5_total_data/{CLS}/labels/{i}', f'./6_tmp_img/labels/{i}')

In [25]:
for CLS in ['train', 'test']:
    
    txt_list = os.listdir(f'./5_total_data/{CLS}/labels')
    
    for txt in txt_list:
            
            with open(f'./5_total_data/{CLS}/labels/{txt[:-4]}.txt', 'r') as f:
                for x in f.readlines():
                    
                    if x.split()[0] == '17':
                        print('왜 있는데 니가')